# Misc 1 - actualbeepboop

> This time its actually a real beepboop challenge
> 
> - 9V1BD
>
> Files: [output.wav](output.wav)

Alright, a miscellaneous WAV file. Let's see what Audacity has to say about it.

![](audacity1.png)

From looking around a bit, we see a 12.16s waveform that has a very flat timbre. Its frequency changes every 0.02s (or multiple thereof), and it appears to only take four different frequencies.

Let's actually get this down with proper FFTs and stuff. Since it's sampled at 48KHz, we want to capture every $48000 \times 0.02 = 960$ samples.

In [1]:
import numpy as np
import struct

bs = open('output.wav', 'rb').read()[0x2C:] # ignore WAVE metadata
vals = np.array(struct.unpack(len(bs)//2*'h', bs))

arr = []
for i in range(0, len(vals), 960):
    chunk = vals[i:i+960]
    windowed_chunk = chunk * np.hanning(len(chunk))
    fft_data = np.fft.rfft(windowed_chunk)
    freqs = np.fft.rfftfreq(len(chunk), d=1/48000)
    dominant_freq = freqs[np.argmax(np.abs(fft_data))]
    arr.append(dominant_freq)
    
print(f'{set(arr) = }')
print(f'{len(arr) = }')

set(arr) = {2400.0, 3050.0, 2050.0, 2700.0}
len(arr) = 608


Let's check that some things are in order. Firstly, we do have exactly four different frequencies as expected (and they're very nice numbers too, a confirmation that we're going things right). Secondly, we have recorded 608 of these frequencies, and since these are taken over 0.02s, it's another quick check that $608 \times 0.02 = 12.16$ is the length of the clip in seconds.

Alright, let's convert this to base 4 and see if we can spot anything out of it.

In [2]:
''.join(str([2050.0, 2400.0, 2700.0, 3050.0].index(x)) for x in arr)

'30303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303121203111211231323120213030303000312021003000300031331131311030013102213311013033110131003023213030303133113030113000301230312121212221003012302031212122210030123022213121133131212031112112313231202130303030003120210030003000313311313110300131022133110130331101310030232130303031331130301130003012303121212122210030123020312121222100301230222131211331'

Okay, so it seems like it splits into three parts: There's a prelude at the beginning, and then possibly the flag encoded twice?

```
303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303030303

31212031112112313231202130303030003120210030003000313311313110300131022133110130331101310030232130303031331130301130003012303121212122210030123020312121222100301230222131211331

31212031112112313231202130303030003120210030003000313311313110300131022133110130331101310030232130303031331130301130003012303121212122210030123020312121222100301230222131211331
```

At this point there are quite a few ways you could encode the flag, so let's maybe compare with what we expect the first few bytes of the flag to be (hoping it's base 4):

In [3]:
for i, c in enumerate(b'grey{'):
    print(i, chr(c), np.base_repr(c, 4))

0 g 1213
1 r 1302
2 e 1211
3 y 1321
4 { 1323


Hey, that looks _a lot_ like the start of the message `3121 2031 1121 1231 3231`, just in reverse. Guess we can script this a bit.

In [4]:
msg = '31212031112112313231202130303030003120210030003000313311313110300131022133110130331101310030232130303031331130301130003012303121212122210030123020312121222100301230222131211331'
int(msg[::-1],4).to_bytes(len(msg)//4, 'little')

b'grey{b33pb00p_w1th_4_t0n3s_3509gfj09rfj09jg}'